# Synthesize Responses from the Retrieved Contexts

## Step 0: Environment Setup

In [1]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph
from libs import create_vector_index, get_entities,embed_entity, vector_search,enhanced_chunk_finder
import pandas as pd
from conn import connect2Googlesheet
from openai import OpenAI
from synthlibs import graph_retriever, parse_relationships, final_context_builder,synthesize_response

# plot figures in high resolution
%config InlineBackend.figure_format = 'retina'
# Force reload of the .env file
try:
    load_dotenv()
    print("The .env file has been loaded")
except Exception as e:
    print("The .env file has not been loaded. Reported error: ", e)

The .env file has been loaded


In [2]:
# Connect to Neo4j database
try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URL"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD"),
    )
    print("Connected to Neo4j database successfully.")
except ValueError as e:
    print(f"Could not connect to Neo4j database: {e}")

# Check if the entities index exists
index_name = "entities"
query = "SHOW INDEXES YIELD name, type WHERE type = 'VECTOR' AND name = $index_name"

result = graph.query(query, params={"index_name": index_name})
if result:
    print("The 'entities' index already exists.")
else:
    create_vector_index(graph, "entities")

Connected to Neo4j database successfully.
The 'entities' index already exists.


## Step 1: Connect to Google Sheet

In [3]:
spreadsheet = connect2Googlesheet()

# Select the worksheet: relevance
Q_worksheet = spreadsheet.get_worksheet(2)
# Select the worksheet: revised summary
Sum_worksheet = spreadsheet.get_worksheet(6)

# Get all records as a list of dictionaries
ques_data = Q_worksheet.get_all_records()
sum_data = Sum_worksheet.get_all_records()

# Convert to Pandas DataFrame
df_MedQ = pd.DataFrame(ques_data)
df_MedSum = pd.DataFrame(sum_data)

# Display the first few rows of both DataFrames
display(df_MedQ.head())
display(df_MedSum.head())

,condition,number,docs,Question,Mahmud's Note,status,comments,
0,ARDS,1,ACURASYS,Does early administration of neuromuscular blo...,Like,,,
1,ARDS,2,ACURASYS,Do patients with severe ARDS being treated wit...,Replace,fixed,,
2,ARDS,3,ROSE,"In patients with moderate to severe ARDS, does...",Maybe this question: In patients with moderate...,fixed,,
3,ARDS,4,ROSE,Do patients with moderate-to-severe ARDS have ...,Local question (not sure if this is the aim of...,fixed,Wrong concept since PEEP by itself is mandator...,Does the use of neuromuscular blockers in pati...
4,ARDS,5,FACTT,"Among patients with ALI/ARDS, does a conservat...",Local question (not sure if this is the aim of...,fixed,Check if studies defined conservative by CVP <...,


,Doc No,Confirmed by,Topic,Name,Study Design and Methodology,Study Population,Interventions,Comparator,Outcomes,Strengths and Weaknesses,Key Findings and Conclusion
0,1,Keon,ARDS,ACURASYS,"- Multicenter, randomized, double-blind, place...",The study enrolled 340 patients from 20 ICUs i...,- Cisatracurium Group:\n - 3-ml rapid IV infu...,- Placebo group receiving identical IV infusio...,Primary Outcome:\n- 90-day in-hospital mortali...,Strengths:\n- Multicenter design\n- Blinded ra...,Key Findings:\n- Adjusted 90-day survival impr...
1,2,Keon,ARDS,ROSE,"- Multicenter, unblinded, randomized clinical ...",- Patients with moderate-to-severe ARDS\n- Cri...,- 48-hour continuous infusion of cisatracurium...,- Usual care approach\n- No routine neuromuscu...,Primary Outcome:\n- In-hospital death from any...,Strengths:\n- Multicenter design\n- High proto...,Key Findings:\n No significant difference in 9...
2,3,Keon,ARDS,FACTT,"- Prospective, randomized clinical trial\n- Co...","- 1,001 patients with acute lung injury (503 i...",Conservative Strategy:\n- Lower fluid intake\n...,The two fluid management strategies were compa...,Primary Outcome:\n- Death at 60 days\n\nSecond...,Strengths:\n- Randomized design\n- Multi-cente...,Key Findings:\n- No significant difference in ...
3,4,Keon,ARDS,ARDSNet,"- Multicenter, randomized controlled trial\n- ...",- Total Participants: 861 patients\n- Inclusio...,- Lower Tidal Volume Group:\n - Initial tidal...,- Traditional Tidal Volume Group:\n - Initial...,Primary Outcomes:\n- Mortality before hospital...,Strengths:\n- Multicenter design\n- Robust ran...,Key Findings:\n- Mortality reduced by 22% in l...
4,5,Keon,ARDS,PROSEVA,"- Multicenter, prospective, randomized, contro...",- 466 patients with severe ARDS\n- Inclusion c...,- Prone group (237 patients):\n - Turned to p...,- Supine group (229 patients):\n - Remained i...,Primary Outcome:\n- 28-day mortality\n\nSecond...,Strengths:\n- Multicenter design\n- Standardiz...,Key Findings:\n- 28-day mortality: 16.0% (pron...


## Step 2: Build Context before Synthesis

In [18]:
test_question = df_MedQ["Question"][10]
context = graph_retriever(graph, test_question, max_depth=2)
formatted_relations = parse_relationships(context)
print(formatted_relations)

Query: Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?
Found 45 unique relationships:
- patients -> blinded_to -> treatment allocation
- patients -- blinded_to -- 12-month data collectors
- patients -- blinded_to -- study statistician
- Liverpool ICU -> data_collected_from -> patients
- patients -- data_submitted_to -- Australian & New Zealand Intensive Care Society database
- patients -- collected_data -- cardiac arrest patients
- patients -- on_call_at -- cardiologist
- patients -- treated_in -- cardiac arrest patients
- patients -> randomly_assigned_to -> receive haloperidol
- patients -> hospitalized_for -> medical and surgical treatments
- Fluid and vasopressor management -> is_related_to -> septic shock
- acute kidney injury -> has_complication -> septic shock
- septic shock -- managed_by -- renal-replacement therapy
- clinicians -> diagnosed -> septic shock
- septic shock -- challenged_by -

In [19]:
final_context, filenames = final_context_builder(graph, test_question, method="hybrid")
# res = get_firstk_chunks(graph, filenames, firstk=3)
print(final_context)

Query: Patients with septic shock undergoing mechanical ventilation, did continuous infusion of hydrocortisone result in lower 90-day mortality?
Found 45 unique relationships:
- patients -> blinded_to -> treatment allocation
- patients -- blinded_to -- 12-month data collectors
- patients -- blinded_to -- study statistician
- Liverpool ICU -> data_collected_from -> patients
- patients -- data_submitted_to -- Australian & New Zealand Intensive Care Society database
- patients -- collected_data -- cardiac arrest patients
- patients -- on_call_at -- cardiologist
- patients -- treated_in -- cardiac arrest patients
- patients -> randomly_assigned_to -> receive haloperidol
- patients -> hospitalized_for -> medical and surgical treatments
- Fluid and vasopressor management -> is_related_to -> septic shock
- acute kidney injury -> has_complication -> septic shock
- septic shock -- managed_by -- renal-replacement therapy
- clinicians -> diagnosed -> septic shock
- septic shock -- challenged_by -

## Step 3: Synthesize

### 3.1 Stance Synthesis

In [20]:
response_hy = synthesize_response(
    graph,
    test_question,
    synth_type="stance_synthesis",
    method="hybrid",
    model="gpt-3.5-turbo",
)
# response_vec = synthesize_response(
#     graph,
#     test_question,
#     synth_type="stance_synthesis",
#     method="vector",
#     model="gpt-3.5-turbo",
# )
# response_gr= synthesize_response(graph, test_question, method="graph", model="gpt-3.5-turbo")

In [21]:
print("Hybrid Response: ", response_hy.choices[0].message.content.strip())
# print("Vector Response: ", response_vec.choices[0].message.content.strip())
# print("Graph Response: ", response_gr.choices[0].message.content.strip())

Hybrid Response:  {
  "PLMALDHSS": "Supports",
  "TRISS": "Neutral",
  "ADRENAL": "Against",
  "APROCCHSS": "Supports",
  "FRESHS": "Neutral",
  "ANNANE": "Supports",
  "SS3vsSS2": "Neutral",
  "HYPRESS": "Supports",
  "CORTICUS": "Against"
}


### 3.2 PICO Synthesis

In [22]:
response_hy = synthesize_response(
    graph,
    test_question,
    synth_type="PICO_synthesis",
    method="hybrid",
    model="gpt-3.5-turbo",
)

"""
response_vec = synthesize_response(
    graph,
    test_question,
    synth_type="PICO_synthesis",
    method="vector",
    model="gpt-3.5-turbo",
)
"""

'\nresponse_vec = synthesize_response(\n    graph,\n    test_question,\n    synth_type="PICO_synthesis",\n    method="vector",\n    model="gpt-3.5-turbo",\n)\n'

In [23]:
print("Hybrid Response: ", response_hy.choices[0].message.content.strip())
# print("Vector Response: ", response_vec.choices[0].message.content.strip())

Hybrid Response:  Based on the articles provided, the studies can be grouped as follows:

### Supporting the Claim:
1. **Study 1: PLMALDHSS**
   - **Authors:** Romain Pirracchio et al.
   - **Year:** 2023
   - **Title:** Patient-Level Meta-Analysis of Low-Dose Hydrocortisone in Adults with Septic Shock
   - **Journal:** NEJM Evidence
   - **Research Question:** To evaluate the effect of hydrocortisone infusion on 90-day mortality in patients with septic shock.
   - **Study Design:** Meta-analysis of patient-level data from septic shock trials.
   - **Population Size:** 528 participants across 17 trials.
   - **Intervention:** Hydrocortisone infusion.
   - **Primary Outcome:** 90-day mortality was not significantly reduced by hydrocortisone.
   - **Key Findings:** Hydrocortisone did not result in lower 90-day mortality compared to placebo.
   - **Conclusion:** The study findings do not support the use of hydrocortisone to reduce 90-day mortality in septic shock patients.

### Refuting t

In [10]:
response_hy = synthesize_response(
    graph,
    test_question,
    synth_type="PICO_synthesis",
    method="hybrid",
    model="gpt-3.5-turbo",
)

In [11]:
print("Hybrid Response: ", response_hy.choices[0].message.content.strip())

Hybrid Response:  ### Supporting the Claim:
1. **Study 1: FMWSCPARDS**
    - **Authors:** Grissom et al.
    - **Year:** 2015
    - **Title:** FMWSCPARDS
    - **Journal:** Critical Care Medicine
    - **Research Question:** Investigate the impact of different fluid management protocols on outcomes in ARDS patients.
    - **Type of Study:** Retrospective comparison
    - **Methodology:** Compared FACTT Lite, FACTT Conservative, and FACTT Liberal protocols in ARDS patients.
    - **Population Size:** 5003 participants across different management protocols.
    - **Interventions:** Fluid management by protocol.
    - **Outcomes:** Equivalent ventilator-free days in FACTT Lite and FACTT Conservative groups.
    - **Key Findings:** FACTT Lite protocol had equivalent ventilator-free days to FACTT Conservative.
    - **Conclusion:** FACTT Lite protocol is safe and has similar outcomes to FACTT Conservative.

### Supporting the Claim:
2. **Study 2: SMDLTSWCI**
    - **Authors:** Not mentioned

In [16]:
response_hy = synthesize_response(
    graph,
    test_question,
    synth_type="PICO_synthesis",
    method="hybrid",
    model="gpt-4o-mini",
)

In [17]:
print("Hybrid Response: ", response_hy.choices[0].message.content.strip())

Hybrid Response:  Based on the provided articles, I have categorized the studies according to their findings regarding the claim that early administration of neuromuscular blocking agents (NMBAs) increases ventilator-free days (VFDs):

### Supporting the Claim
1. **Authors**: Lyu et al.
   - **Year**: 2021
   - **Title**: The effect of neuromuscular blocking agents uses in acute respiratory distress syndrome: a systematic review and meta-analysis of randomized controlled trials.
   - **Journal**: Minerva Anestesiologica
   - **Research Question**: Does the early administration of NMBAs improve ventilator-free days in patients with moderate to severe ARDS?
   - **Type of Study**: Meta-analysis of randomized controlled trials.
   - **Methodology**: Reviewed multiple RCTs comparing early NMBA use versus control in ARDS patients, focusing on VFDs.
   - **Population Size**: Data from numerous trials with a total of 1,953 patients.
   - **Key Findings**: Early NMBA infusion was associated wi